In [27]:
using Mmap, HDF5, SQLite, CSV, Tables, DataFrames, Suppressor

In [28]:
file = "ratings.csv"
bigfile = "/Users/guillaume/Downloads/ml-latest/ratings.csv"

"/Users/guillaume/Downloads/ml-latest/ratings.csv"

In [29]:
csv1 = CSV.File(file, use_mmap=true)

CSV.File("ratings.csv", rows=100836):
Tables.Schema:
 :userId     Union{Missing, Int64}  
 :movieId    Union{Missing, Int64}  
 :rating     Union{Missing, Float64}
 :timestamp  Union{Missing, Int64}  

## SQLite

In [30]:
db = SQLite.DB("benchmark.sqlite")
csv1 |> SQLite.load!(db, "ratings")

"ratings"

In [31]:
typeof(db)

SQLite.DB

In [32]:
@time begin
    res = @suppress_err SQLite.query(db, "SELECT userId, movieId as itemId, rating as value FROM ratings ORDER BY userId ASC LIMIT 1000 OFFSET 1000")
    res[[:itemId, :value]]
end

  0.055605 seconds (7.20 k allocations: 430.007 KiB)


,itemId,value
,Int64⍰,Float64⍰
1,3897,4.0
2,3911,3.0
3,3967,4.0
4,3996,4.0
5,4002,3.0
6,4014,1.0
7,4020,3.0
8,4021,4.0
9,4027,3.0


In [33]:
@suppress_err SQLite.query(db, "SELECT AVG(rating) as mean from ratings")

,mean
,Float64⍰
1,3.50156


In [34]:
varinfo()

| name        |       size | summary                                                                                                                                                                                                                                                                                                                      |
|:----------- | ----------:|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| Base        |            | Module                                                                                                                                                                                                                                                                                                                       |
| Core        |            | Module                                                                                                                                                                                                                                                                                                                       |
| Main        |            | Module                                                                                                                                                                                                                                                                                                                       |
| bigfile     |   56 bytes | String                                                                                                                                                                                                                                                                                                                       |
| csv1        |  3.138 MiB | CSV.File{false,true,Base.GenericIOBuffer{Array{UInt8,1}},Parsers.Delimited{false,true,Parsers.Quoted{Parsers.Strip{Parsers.Sentinel{typeof(Parsers.defaultparser),Parsers.Trie{0x00,false,missing,2,Tuple{}}}}},Parsers.Trie{0x00,false,missing,8,Tuple{Parsers.Trie{0x2c,true,missing,8,Tuple{}}}}},NamedTuple{(),Tuple{}}} |
| db          |   48 bytes | SQLite.DB                                                                                                                                                                                                                                                                                                                    |
| file        |   19 bytes | String                                                                                                                                                                                                                                                                                                                       |
| res         | 37.445 KiB | 1000×3 DataFrame                                                                                                                                                                                                                                                                                                             |
| startupfile |   56 bytes | String                                                                                                                                                                                                                                                                                                                       |


In [35]:
rm("benchmark.sqlite")

## Mmap

In [36]:
csv2 = CSV.File(file, use_mmap=true)

CSV.File("ratings.csv", rows=100836):
Tables.Schema:
 :userId     Union{Missing, Int64}  
 :movieId    Union{Missing, Int64}  
 :rating     Union{Missing, Float64}
 :timestamp  Union{Missing, Int64}  

In [37]:
tempfile = tempname()

"/var/folders/yq/8t7k_n_d4b39wmbl31mvkdgm0000gn/T/juliasNjtvG"

In [38]:
A = Mmap.mmap(tempfile, Array{Tuple{Int64, Int64, Float64},1}, length(csv2))

for (i,r) in enumerate(csv2)
    A[i] = (r.userId, r.movieId, r.rating)
end

Mmap.sync!(A)

@time begin
    sort(A, by=r->r[2])
end

  0.096953 seconds (162.45 k allocations: 11.752 MiB)


100836-element Array{Tuple{Int64,Int64,Float64},1}:
 (1, 1, 4.0)       
 (5, 1, 4.0)       
 (7, 1, 4.5)       
 (15, 1, 2.5)      
 (17, 1, 4.5)      
 (18, 1, 3.5)      
 (19, 1, 4.0)      
 (21, 1, 3.5)      
 (27, 1, 3.0)      
 (31, 1, 5.0)      
 (32, 1, 3.0)      
 (33, 1, 3.0)      
 (40, 1, 5.0)      
 ⋮                 
 (338, 190221, 1.0)
 (184, 191005, 4.5)
 (184, 193565, 3.5)
 (184, 193567, 3.0)
 (184, 193571, 4.0)
 (184, 193573, 4.0)
 (184, 193579, 3.5)
 (184, 193581, 4.0)
 (184, 193583, 3.5)
 (184, 193585, 3.5)
 (184, 193587, 3.5)
 (331, 193609, 4.0)

In [39]:
varinfo()

| name        |       size | summary                                                                                                                                                                                                                                                                                                                      |
|:----------- | ----------:|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| A           |  2.308 MiB | 100836-element Array{Tuple{Int64,Int64,Float64},1}                                                                                                                                                                                                                                                                           |
| Base        |            | Module                                                                                                                                                                                                                                                                                                                       |
| Core        |            | Module                                                                                                                                                                                                                                                                                                                       |
| Main        |            | Module                                                                                                                                                                                                                                                                                                                       |
| bigfile     |   56 bytes | String                                                                                                                                                                                                                                                                                                                       |
| csv1        |  3.138 MiB | CSV.File{false,true,Base.GenericIOBuffer{Array{UInt8,1}},Parsers.Delimited{false,true,Parsers.Quoted{Parsers.Strip{Parsers.Sentinel{typeof(Parsers.defaultparser),Parsers.Trie{0x00,false,missing,2,Tuple{}}}}},Parsers.Trie{0x00,false,missing,8,Tuple{Parsers.Trie{0x2c,true,missing,8,Tuple{}}}}},NamedTuple{(),Tuple{}}} |
| csv2        |  3.138 MiB | CSV.File{false,true,Base.GenericIOBuffer{Array{UInt8,1}},Parsers.Delimited{false,true,Parsers.Quoted{Parsers.Strip{Parsers.Sentinel{typeof(Parsers.defaultparser),Parsers.Trie{0x00,false,missing,2,Tuple{}}}}},Parsers.Trie{0x00,false,missing,8,Tuple{Parsers.Trie{0x2c,true,missing,8,Tuple{}}}}},NamedTuple{(),Tuple{}}} |
| db          |   48 bytes | SQLite.DB                                                                                                                                                                                                                                                                                                                    |
| file        |   19 bytes | String                                                                                                                                                                                                                                                                                                                       |
| res         | 37.445 KiB | 1000×3 DataFrame                                                                                                                                                                                                                                                                                                             |
| startupfile |   56 bytes | String                                                                                                                                                                                                                                                                                                                       |
| tempfile    |   68 bytes | String                                                                                                                                                                                                                                                                                                                       |


In [40]:
rm(tempfile)